<a href="https://colab.research.google.com/github/myjjojjojjo/Lesson/blob/main/12_%EC%9E%90%EB%8F%99%EC%B0%A8_%EA%B0%9D%EC%B2%B4_%EC%B6%94%EC%B6%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. yolov5 다운로드 및 환경 설정

In [ ]:
# git에서 yolov5 모델 다운로드
!git clone https://github.com/ultralytics/yolov5

In [ ]:
# 기본 환경 세팅
%cd yolov5
!pip install -qr requirements.txt

In [ ]:
# 모듈 import
import torch
from IPython.display import Image, clear_output, display
from glob import glob
import numpy as np
import cv2
import random
import yaml

In [ ]:
# 구글 드라이브 연결
from google.colab import drive
drive.mount('/content/drive')

2. 데이터 다운로드
- Vehicles 폴더 생성후 데이터 저장

In [ ]:
# 폴더 생성 및 이동
%mkdir /content/yolov5/Vehicles
%cd /content/yolov5/Vehicles

In [ ]:
# 데이터 다운로드 및 압축 해제
!curl -L "https://public.roboflow.com/ds/AJq9ZY0PrE?key=zxAFjSPiMG" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip

In [ ]:
# 저장 데이터 확인
!ls

In [ ]:
!ls ./test

3. 데이터 목록 생성
- test/train/valid 데이터 목록 생성(*.txt)

In [ ]:
# train/test/valid 이미지 파일 목록 생성
train_img_list=glob('./train/images/*.jpg')
test_img_list=glob('./test/images/*.jpg')
valid_img_list=glob('./valid/images/*.jpg')

print(len(train_img_list),len(test_img_list),len(valid_img_list))

In [ ]:
%pwd

In [ ]:
# 이미지 목록을 이용해 txt 파일 생성(Vehicles 폴더에 저장)
with open('./train.txt', 'w') as f:
    f.write('\n'.join(train_img_list)+'\n')

with open('/content/yolov5/Vehicles/test.txt', 'w') as f:
    f.write('\n'.join(test_img_list)+'\n')

with open('/content/yolov5/Vehicles/valid.txt', 'w') as f:
    f.write('\n'.join(valid_img_list)+'\n')

%ls

4. 데이터 정보 수정(data.yaml)
- jupyter, 메모장, 코드 등을 이용해 수정

In [ ]:
# 개별 수정후 저장된 코드 확인
%cat /content/yolov5/Vehicles/data.yaml

5. 학습 모델(./models/yolov5s.yaml 모델) 구성 편집 후 확인
- jupyter, 메모장, 코드 등을 이용해 수정
- 사용 모델의 nc 값을 data.yaml의 nc와 동일하게 수정

In [ ]:
# 수정된 학습 모델 확인(nc 확인)
%cat /content/yolov5/models/yolov5s.yaml

6. 학습 진행
* train.py 를 이용해 학습
    * `img`: 입력 이미지 크기 정의
    * `batch`: 배치 크기 결정
    * `epochs`: 학습 기간 개수 정의
    * `data`: yaml 파일 경로
    * `cfg`: 모델 구성 지정
    * `weights`: 가중치에 대한 경로 지정
    * `name`: 결과 이름
    * `nosave`: 최종 체크포인트만 저장
    * `cache`: 빠른 학습을 위한 이미지 캐시

In [ ]:
# 작업 위치를 yolov5 폴더로 변경
%cd /content/yolov5

In [ ]:
# python train.py을 이용해 학습 진행
!python train.py --img 416 --batch 32 --epochs 100 --data ./Vehicles/data.yaml --cfg ./models/yolov5s.yaml --weight '' --name Vehicels_results --cache

7. 학습 진행 상황 확인

In [ ]:
Image(filename='/content/yolov5/runs/train/Vehicels_results/results.png', width=800)

In [ ]:
%load_ext tensorboard
%tensorboard --logdir /content/yolov5/runs/

8. 학습 및 검증 데이터(이미지) 확인

In [ ]:
Image(filename='/content/yolov5/runs/train/Vehicels_results/train_batch0.jpg', width=800)

In [ ]:
Image(filename='/content/yolov5/runs/train/Vehicels_results/val_batch0_pred.jpg', width=800)

9. 테스트 데이터를 이용한 검증
- val.py 파일 실행을 통해 확인
- valid/test 데이터 이용

In [ ]:
# valid 데이터 파일을 이용한 검증
!python val.py --weights /content/yolov5/runs/train/Vehicels_results/weights/best.pt --data /content/yolov5/Vehicles/data.yaml

In [ ]:
Image(filename='./runs/val/exp/val_batch0_pred.jpg', width=800)

In [ ]:
# test 이미지를 이용한 검증
!python val.py --weights ./runs/train/Vehicels_results/weights/best.pt --data ./Vehicles/data.yaml --task test

In [ ]:
Image(filename='./runs/val/exp2/val_batch0_pred.jpg', width=800)

10. 모델을 이용한 이미지에 Detect 적용
- detect.py 이용

 - test 이미지를 이용한 확인

In [ ]:
!python detect.py --weights ./runs/train/Vehicels_results/weights/best.pt --source ./Vehicles/test/images

In [ ]:
# detect 결과중 임의로 하나 출력
Image(filename=random.choice(glob('./runs/detect/exp/*.jpg')))

- 사용자 이미지를 이용한 확인

In [ ]:
#사용자 파일 업로드
from google.colab import files

filename=files.upload()

In [ ]:
!python detect.py --weights ./runs/train/Vehicels_results/weights/best.pt --source ./data/images/test_img1.jpg

In [ ]:
Image(filename='./runs/detect/exp2/test_img1.jpg')

- 사용자 동영상 확인

In [ ]:
!python detect.py --weights ./runs/train/Vehicels_results/weights/best.pt --source /content/drive/MyDrive/hkedu/data/test_thans1.mp4

In [ ]:
!ls ./runs/detect/exp3

In [ ]:
# jupyter에서 실행
#videocap=cv2.VideoCapture('./runs/detect/exp3/test_thans1.mp4')

#while True:
#    ret, frame = videocap.read()
#    cv2.imshow('Vehicels Cap',frame)
#    if cv2.waitKey(10) == 27:
#        break

#videocap.release()
#cv2.destroyAllWindows()

In [ ]:
files.download('/content/yolov5/runs/detect/exp3/test_thans1.mp4')

In [ ]:
# 모델 다운로드
files.download('/content/yolov5/runs/train/Vehicels_results/weights/best.pt')